This notebook goes through the schemas.yml file and attempts to guess the list of input files for each script and provide hints for the `scripts.yml`.

In [17]:
import cea.config
import cea.inputlocator
import cea.scripts
import yaml
import os
import inspect

In [19]:
config = cea.config.Configuration()
locator = cea.inputlocator.InputLocator(scenario=config.scenario)

In [13]:
# these are the input files we are working on
cea_folder = os.path.join(os.path.dirname(cea.config.__file__))
scripts_yml = os.path.join(cea_folder, 'scripts.yml')
schemas_yml = os.path.join(cea_folder, 'schemas.yml')

In [14]:
schemas = yaml.load(open(schemas_yml))

In [15]:
# collect a list of input_files used by each script known to schemas.yml
scripts = {}
for k in schemas.keys():
    if 'used_by' in schemas[k]:
        for s in schemas[k]['used_by']:
            scripts.setdefault(s, []).append(k)

In [23]:
# test to see how we can figure out the "easy" methods
for mname in scripts['data-helper']:
    method = getattr(locator, mname)
    print inspect.getargspec(method).args

['self']
['self']
['self']
['self']


In [28]:
for script in scripts.keys():
    print script
    for mname in scripts[script]:
        method = getattr(locator, mname)
        args = inspect.getargspec(method).args[1:]   # skip self
        args = [a for a in args if not a in {'format'}]
        print '  - [{ifdata}]'.format(ifdata=', '.join([mname] + args))

sewage-potential
  - [get_total_demand]
  - [get_demand_results_file, building_name]
data-helper
  - [get_building_age]
  - [get_building_occupancy]
  - [get_archetypes_schedules]
  - [get_archetypes_properties]
network-layout
  - [get_street_network]
  - [get_total_demand]
  - [get_zone_geometry]
operation-costs
  - [get_total_demand]
  - [get_building_supply]
  - [get_life_cycle_inventory_supply_systems]
solar-collector
  - [get_radiation_metadata, building_name]
  - [get_zone_geometry]
  - [get_radiation_building, building_name]
  - [get_supply_systems]
  - [get_weather, name]
thermal-network
  - [get_network_layout_nodes_shapefile, network_type, network_name]
  - [get_demand_results_file, building_name]
  - [get_thermal_networks]
  - [get_supply_systems]
  - [get_weather, name]
photovoltaic-thermal
  - [get_radiation_metadata, building_name]
  - [get_zone_geometry]
  - [get_radiation_building, building_name]
  - [get_supply_systems]
  - [get_weather, name]
emissions
  - [get_total_

some problematic methods:

- get_radiation_building(building_name)
  - check for the first building in the zone?
  - use sequence of input files to ensure that the zone exists?
  - (or just special case it?)
  - also:
    - get_radiation_metadata(building_name)
- get_network_layout_nodes_shapefile(network_type, network_name)
  - network_type, network_name are special and dependent on the script used
  - solution: include fqnames as arguments

In [27]:
yaml.load(open(scripts_yml))['Energy potentials']

[{'description': 'Calculate the heat extracted from the Lake',
  'interfaces': ['cli', 'arcgis', 'dashboard'],
  'label': 'Lake potential',
  'module': 'cea.resources.lake_potential',
  'name': 'lake-potential',
  'parameters': ['general:scenario', 'lake']},
 {'description': 'Calculate electricity & heat production from photovoltaic / thermal technologies',
  'interfaces': ['cli', 'arcgis', 'dashboard'],
  'label': 'Photovoltaic-thermal Panels',
  'module': 'cea.technologies.solar.photovoltaic_thermal',
  'name': 'photovoltaic-thermal',
  'parameters': ['general:scenario',
   'general:weather',
   'general:multiprocessing',
   'general:number-of-cpus-to-keep-free',
   'solar:buildings',
   'solar:type-pvpanel',
   'solar:type-scpanel',
   'solar:panel-on-roof',
   'solar:panel-on-wall',
   'solar:annual-radiation-threshold',
   'solar:solar-window-solstice',
   'solar:t-in-pvt']},
 {'description': 'Calculate electricity production from solar photovoltaic technologies',
  'interfaces': 